In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

excutable_path = 'C:/Users/Flora/datapython/chromedriver (3) 98.047v.exe'

source_url = "https://map.kakao.com/"

driver = webdriver.Chrome(executable_path=excutable_path)
driver.get(source_url)

searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
searchbox.send_keys("창원 맛집") #검색창 이용

searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") #검색버튼 누름
driver.execute_script("arguments[0].click();", searchbutton)

time.sleep(2) #기다리기

html = driver.page_source #결과소스

soup = BeautifulSoup(html, "html.parser")
moreviews = soup.find_all(name="a", attrs={"class":"moreview"})

page_urls = []
for moreview in moreviews:
    page_url = moreview.get("href")
    print(page_url)
    page_urls.append(page_url)

driver.close()

https://place.map.kakao.com/1492586965
https://place.map.kakao.com/2110248909
https://place.map.kakao.com/26904719
https://place.map.kakao.com/26798365
https://place.map.kakao.com/631239859
https://place.map.kakao.com/21959844
https://place.map.kakao.com/1181055498
https://place.map.kakao.com/437541294
https://place.map.kakao.com/8059361
https://place.map.kakao.com/1012202600
https://place.map.kakao.com/19749793
https://place.map.kakao.com/1187561607
https://place.map.kakao.com/12660894
https://place.map.kakao.com/1902526523
https://place.map.kakao.com/8764226


In [3]:
columns = ['score', 'review']
df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path=excutable_path)  # for Windows
for page_url in page_urls:
    
    driver.get(page_url) #상세보기 페이지
    time.sleep(2)
    
    html = driver.page_source #첫페이지
    soup = BeautifulSoup(html, 'html.parser')
    contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
    
    #별점
    rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
    
    #리뷰
    reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
    
    for rate, review in zip(rates, reviews):
        row = [rate.text[0], review.find(name="span").text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)
    
    for button_num in range(2, 10):
        try:
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            another_reviews.click()
            time.sleep(2)
            
            #리뷰페이지
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})

            #별점
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})

            #리뷰
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

            for rate, review in zip(rates, reviews):
                row = [rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=df.columns)
                df = df.append(series, ignore_index=True)
        except:
            break    
driver.close()

In [4]:
df['y'] = df['score'].apply(lambda x: 1 if float(x) > 3 else 0)
print(df.shape)
df.head()

(371, 3)


,score,review,y
0,3,어시장과 일반횟집을 합쳐놓은듯…가성비는 좋음,0
1,5,포장 기본 1시간 맛난건 오전에 품절,1
2,2,가성비는 좋지만 ~ 너무 시끄럽고 정신없는 듯.. 회맛을 모르고 후딱 먹고 빨리 ...,0
3,5,가성비 최고! 소문듣고 갔는데 진짜네요. 예약 필수,1
4,4,"스끼 필요 없고, 저렴한 가격에 회 많이 먹고 싶다? 무조건입니다. 매운탕은 삼삼...",1


In [5]:
df.to_csv("review_data_v4.csv", index=False)

In [6]:
df = pd.read_csv("review_data_v4.csv")

In [7]:
import re

def text_cleaning(text): #한글만 추출
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('',str(text)) 
    return result

In [8]:
df = pd.read_csv("review_data_v4.csv")
df['ko_text'] = df['review'].apply(lambda x: text_cleaning(x))

In [9]:
del df['review']



In [10]:
df = df[df['ko_text'].str.len() > 3]
df.head()

,score,y,ko_text
0,3,0,어시장과 일반횟집을 합쳐놓은듯가성비는 좋음
1,5,1,포장 기본 시간 맛난건 오전에 품절
2,2,0,가성비는 좋지만 너무 시끄럽고 정신없는 듯 회맛을 모르고 후딱 먹고 빨리 나가야...
3,5,1,가성비 최고 소문듣고 갔는데 진짜네요 예약 필수
4,4,1,스끼 필요 없고 저렴한 가격에 회 많이 먹고 싶다 무조건입니다 매운탕은 삼삼하게 ...


In [11]:
idx_sc1 = df[df['score'] == 3].index
df = df.drop(idx_sc1)

In [12]:
# idx_sc2 = df[df['score'] == 4].index
# df = df.drop(idx_sc2)

In [13]:
print(df.shape)

(244, 3)


In [14]:
df.head(20)

,score,y,ko_text
1,5,1,포장 기본 시간 맛난건 오전에 품절
2,2,0,가성비는 좋지만 너무 시끄럽고 정신없는 듯 회맛을 모르고 후딱 먹고 빨리 나가야...
3,5,1,가성비 최고 소문듣고 갔는데 진짜네요 예약 필수
4,4,1,스끼 필요 없고 저렴한 가격에 회 많이 먹고 싶다 무조건입니다 매운탕은 삼삼하게 ...
5,5,1,서빙 이모님들 친절하시고 모든매뉴 가성비대비 최고임니다
6,5,1,이곳은 성지입니다 매운탕은 호불호가 갈립니다
7,5,1,저렴하고 쾌적하고 양많고 맛있어요
13,1,0,주차요원이 너무 불친절해요
14,5,1,회가 아주 싱싱하고 가격도 합리적입니다 매운탕도 시원하고 만족
15,5,1,진짜 괜찮습니다 근처면 꼭 가보세요


In [15]:
from konlpy.tag import Okt

def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word, tag) for word, tag in pos]
    return pos

result = get_pos(df['ko_text'].values[0])
print(result)

['포장/Noun', '기본/Noun', '시간/Noun', '맛/Noun', '난건/Noun', '오전/Noun', '에/Josa', '품절/Noun']


In [16]:
kk = []

In [17]:
# kk = ['우동/Noun', '튀김/Noun', '고기/Noun', '오리/Noun', '타코/Noun', '불고기/Noun', '창원/Noun']

In [18]:
# search = "/Noun"
# for i in range(203):
#     for word in get_pos(df['ko_text'].values[i]):
#         if search in word: 
#             kk.append(word)  
# print(kk)
# stop_words = kk

In [19]:
kk = ['회맛/Noun', '나가야/Noun', '스끼/Noun', '회/Noun', '매운탕/Noun', '나오니/Noun', '마늘/Noun', '청양고추/Noun', '매운탕/Noun', '회/Noun', '매운탕/Noun', '회도/Noun', '매운탕/Noun', '지난번/Noun', '회/Noun', '회/Noun', '조개/Noun',  '공기밥/Noun', '횟집/Noun', '회맛/Noun', '물회/Noun', '장어/Noun', '탕/Noun', '물회/Noun', '듬회꼬/Noun', '치/Noun', '물회/Noun', '대게/Noun', '게/Noun', '밥/Noun', '점/Noun', '알/Noun', '차/Noun', '횟집/Noun', '회/Noun', '해산물/Noun', '게/Noun', '수족관/Noun', '것/Noun', '완판되는/Noun', '해산물/Noun', '횟집/Noun', '부산/Noun', '전국/Noun', '두루/Noun', '두루/Noun', '부산/Noun', '부산/Noun', '진해/Noun', '진해/Noun', '횟집/Noun', '부회/Noun', '타/Noun', '석화/Noun', '천원/Noun', '만원/Noun', '집앞/Noun', '횟집/Noun', '접수/Noun', '김치/Noun', '중국산/Noun', '밑반찬/Noun', '음식/Noun', '곳/Noun', '가게/Noun', '고기/Noun', '갈비/Noun', '오늘/Noun', '처음/Noun', '것/Noun', '창원/Noun', '손님/Noun', '모시/Noun', '집/Noun', '어디가/Noun', '곳/Noun', '블로그/Noun', '홍보/Noun', '글/Noun', '보고/Noun', '갈비/Noun', '마블링/Noun', '덩어리/Noun', '기름/Noun', '등심/Noun',  '향/Noun', '부위/Noun', '제공/Noun', '게다가/Noun', '냉면/Noun', '면/Noun', '면/Noun', '여행/Noun', '곳/Noun', '가요/Noun', '여행/Noun', '창원/Noun', '집/Noun', '재/Noun', '방문/Noun', '곳/Noun', '어른/Noun', '셋/Noun', '이서/Noun', '우모/Noun', '듬/Noun', '정식/Noun', '인분/Noun', '돼지갈비/Noun', '인분/Noun', '배/Noun', '여기/Noun', '반찬/Noun', '갈비/Noun', '이모/Noun', '오늘/Noun', '고기/Noun', '콜/Noun', '이모/Noun', '가지/Noun', '고기/Noun', '밑반찬/Noun', '짐/Noun', '점심/Noun', '한정식/Noun', '가게/Noun', '규모/Noun', '접객/Noun', '시스템/Noun', '매번/Noun', '휴가/Noun', '때/Noun',  '점심/Noun',  '손님/Noun', '곳/Noun', '느낌/Noun', '제대로/Noun', '코로나/Noun', '안전/Noun', '및/Noun', '예방/Noun', '조치/Noun', '탁월/Noun', '보이/Noun', '모든/Noun',   '제/Noun', '값/Noun', '함/Noun', '음식/Noun', '장소/Noun',    '보장/Noun', '서브/Noun',  '직원/Noun', '밑반찬/Noun', '티/Noun', '고기/Noun', '제일/Noun', '일단/Noun', '벚꽃/Noun', '갈비/Noun', '양념/Noun', '이동/Noun', '갈비/Noun', '집중/Noun', '제일/Noun', '생갈/Noun', '비도/Noun', '소/Noun', '살살/Noun', '식감/Noun', '예전/Noun', '회식/Noun', '자주/Noun', '코로나/Noun', '담번/Noun', '우모/Noun', '듬/Noun',  '옛날/Noun', '점심/Noun', '지금/Noun',  '후/Noun', '덜덜/Noun', '자주/Noun', '상남동/Noun', '손님/Noun', '접대/Noun', '때/Noun', '꼭/Noun', '곳/Noun',  '곳/Noun', '강/Noun', '로타리/Noun', '클럽/Noun', '행사/Noun', '층/Noun', '진행/Noun', '수/Noun', '음식/Noun', '달/Noun', '종류/Noun', '젓가락/Noun', '곳/Noun', '상남동/Noun', '자리/Noun', '값/Noun', '이건/Noun', '평가/Noun',  '듯/Noun', '음식/Noun', '전부/Noun', '달기/Noun', '특색/Noun', '광고/Noun', '완전/Noun', '점심/Noun',  '것/Noun',  '다시/Noun',  '명가/Noun', '수준/Noun',  '일반/Noun',  '수준/Noun', '임/Noun', '안심/Noun',  '아이/Noun', '이집/Noun', '육회/Noun', '매우/Noun', '냉면/Noun', '평양/Noun', '냉면/Noun', '진주/Noun', '하연/Noun', '옥/Noun', '냉면/Noun', '식/Noun', '창원/Noun', '냉면/Noun', '개인/Noun', '가장/Noun', '취향/Noun', '원/Noun', '정식/Noun', '엄마/Noun', '할머니/Noun', '모시/Noun', '두/Noun', '분/Noun',  '괜찬음/Noun', '이/Noun',   '친구/Noun', '고기/Noun', '먼저/Noun', '양념/Noun', '소갈비/Noun', '소갈비/Noun', '무엇/Noun', '반찬/Noun',  '분/Noun', '다음/Noun', '번/Noun', '가족/Noun', '모임/Noun', '여기/Noun', '할머님/Noun', '듯/Noun', '참고/Noun', '불도/Noun', '숯불/Noun', '오늘/Noun',  '갈비/Noun',  '대비/Noun',  '쫌/Noun', '커피/Noun', '티라미수/Noun', '면/Noun', '바싹/Noun',  '곳/Noun',  '티라미수/Noun', '상태/Noun', '뮤/Noun',  '거/Noun', '얘기/Noun', '안/Noun', '배/Noun', '사장/Noun', '리뷰/Noun', '보시/Noun', '조심/Noun',    '딸/Noun', '최애/Noun', '레이스/Noun', '레인보우/Noun', '케잌/Noun', '곳/Noun', '나머진/Noun', '중간/Noun', '정도/Noun', '커피/Noun', '경남/Noun', '엔젤리너스/Noun', '맹탕/Noun', '중간/Noun', '이상은/Noun',  '브라운필드/Noun', '리노/Noun', '베이/Noun', '션/Noun', '브라운/Noun', '핸즈/Noun', '값/Noun',  '맥주/Noun', '팔고/Noun', '연/Noun', '옴/Noun', '염장/Noun', '주의/Noun', '욘/Noun',  '오점/Noun',  '바람/Noun', '카페/Noun', '라떼/Noun', '천원/Noun', '커피/Noun', '낮/Noun', '가면/Noun',   '커피/Noun', '테이블/Noun', '다리/Noun', '높이/Noun', '계속/Noun', '임/Noun', '점원/Noun', '소통/Noun',   '야경/Noun',  '정말/Noun', '예쁘댜/Noun', '음료/Noun', '빙수/Noun',  '반의/Noun', '반도/Noun', '매우/Noun',  '라떼/Noun', '아이스/Noun', '라떼/Noun', '타코/Noun', '내부/Noun', '생각/Noun', '메뉴/Noun', '창원/Noun', '집/Noun', '타코/Noun', '종류/Noun', '메뉴/Noun', '더/Noun', '타코/Noun',  '대비/Noun', '생각/Noun',  '괜춘/Noun',  '나쵸/Noun', '그거/Noun', '배/Noun', '먹음/Noun', '서울/Noun', '자주/Noun', '타코/Noun', '집들이/Noun', '비교/Noun', '때/Noun', '정말/Noun',    '불친절/Noun',   '평범/Noun', '돈/Noun', '안감/Noun', '양도/Noun', '은근/Noun', '사람/Noun',   '독보/Noun', '사람/Noun', '타코/Noun', '타코/Noun', '념념굿/Noun', '생각/Noun', '음식/Noun', '망고/Noun', '타코/Noun', '비프/Noun', '퀘사디아/Noun', '쉬/Noun', '림프/Noun', '중/Noun', '갑/Noun', '망고/Noun', '타코/Noun', '규/Noun', '스테이크/Noun', '위/Noun', '시킴/Noun', '비프/Noun', '타코/Noun', '제일/Noun', '재/Noun', '방문/Noun', '의사/Noun', '프로/Noun', '타코/Noun', '재/Noun', '방문/Noun', '의사/Noun', '토/Noun', '도스/Noun', '메뉴/Noun', '란/Noun', '창원/Noun', '멕시코/Noun', '음식/Noun', '다만/Noun', '향신료/Noun',  '버섯/Noun', '튀김/Noun', '대안/Noun', '주관/Noun', '제/Noun', '입맛/Noun', '이태원/Noun', '멕시코/Noun', '식당/Noun', '더/Noun', '망고/Noun', '쉬/Noun', '림프/Noun', '타코/Noun',  '느낌/Noun', '핫소스/Noun', '규/Noun', '스테이크/Noun', '진짜/Noun', '라이스/Noun', '전체/Noun',  '토핑/Noun', '타코/Noun', '라이스/Noun', '치폴레/Noun',  '양도/Noun', '완전/Noun',  '방역/Noun', '관리/Noun', '제대로/Noun', '정말/Noun', '명부/Noun', '마스크/Noun', '착용/Noun', '셀프/Noun', '바/Noun', '마스크/Noun', '착용/Noun', '요청/Noun', '손소독/Noun', '후/Noun', '입장/Noun', '등/Noun', '에피타이저/Noun', '나초/Noun', '소스/Noun', '토/Noun', '도스/Noun', '그릴/Noun', '드/Noun', '파히타/Noun', '자체/Noun', '또띠아/Noun', '진심/Noun', '배/Noun', '또띠아/Noun', '리필/Noun', '가능/Noun',   '것/Noun', '완벽/Noun', '창원/Noun', '곳/Noun', '멕시칸/Noun', '음식/Noun', '쫀맛입/Noun', '니드/Noun', '음식/Noun', '자주/Noun', '가요/Noun', '타코/Noun', '정통/Noun', '타코/Noun', '한국인/Noun', '입맛/Noun', '타코/Noun', '저/Noun', '더/Noun', '재료/Noun', '음식/Noun',  '등/Noun', '약간/Noun', '편/Noun', '듯/Noun', '타코/Noun', '고기/Noun', '때/Noun', '손님/Noun', '새우/Noun', '고기/Noun', '식/Noun',  '고기/Noun',  '나/Noun', '편/Noun',  '타코/Noun', '감자/Noun', '스프/Noun', '라임/Noun', '모히또/Noun', '꽤/Noun', '동굴/Noun', '동굴/Noun', '내부/Noun', '나름/Noun', '정리/Noun', '팔/Noun', '겉옷/Noun',  '여름/Noun', '입김/Noun', '셀프/Noun', '바/Noun', '동굴/Noun', '미터/Noun', '벨/Noun', '종업원/Noun', '어려움/Noun', '동굴/Noun', '시체/Noun', '동굴/Noun',   '밑반찬/Noun', '샐러드/Noun', '셀프/Noun', '바/Noun', '홀/Noun', '호출/Noun', '버튼/Noun', '오리/Noun',  '남/Noun', '예약/Noun', '다해/Noun', '매장/Noun', '기표/Noun', '손님/Noun', '눈앞/Noun', '일단/Noun', '동굴/Noun', '것/Noun', '자체/Noun', '장점/Noun', '곳/Noun', '정말/Noun', '오리/Noun', '불고기/Noun', '오리탕/Noun', '오리/Noun', '불고기/Noun', '자극/Noun', '오리탕/Noun', '처음/Noun', '닭개장/Noun', '제/Noun', '입/Noun', '오리/Noun', '불고기/Noun', '더/Noun', '오리/Noun', '불고기/Noun', '백숙/Noun', '진짜/Noun',  '특색/Noun', '오리/Noun', '백숙/Noun',  '무엇/Noun',  '남자/Noun', '사장/Noun', '직원/Noun', '일/Noun', '분/Noun', '정말/Noun', '카운터/Noun', '남/Noun', '직원/Noun', '일/Noun', '진짜/Noun', '개애애못함/Noun',  '비도/Noun', '것/Noun', '다만/Noun', '제/Noun', '때/Noun', '워낙/Noun', '묵은지/Noun', '계산/Noun', '때/Noun', '묵은지/Noun', '묵은지/Noun', '찬/Noun', '서빙/Noun', '뭐/Noun', '표정/Noun', '말/Noun', '안/Noun',  '길/Noun',  '단점/Noun', '초밥/Noun', '포장/Noun', '예전/Noun', '포장/Noun', '집/Noun', '스시/Noun', '훨/Noun', '요/Noun', '천안/Noun', '상/Noun', '초밥/Noun', '것/Noun', '오늘/Noun', '초밥/Noun', '시반/Noun', '전/Noun', '예약/Noun', '시/Noun', '분/Noun', '감/Noun', '점심시간/Noun', '층/Noun', '분/Noun', '진짜/Noun',  '정식/Noun', '코스/Noun', '기억/Noun', '창원/Noun', '가면/Noun', '또/Noun', '애오/Noun',  '점점/Noun', '곳/Noun', '곳/Noun', '테이블/Noun', '간격/Noun', '코스/Noun', '서브/Noun', '순서/Noun', '인/Noun', '테이블/Noun', '사시미/Noun', '먹기/Noun', '전/Noun', '초밥/Noun', '서브/Noun', '그릇/Noun', '그릇/Noun', '전/Noun', '다음/Noun', '음식/Noun', '틈/Noun', '모서리/Noun', '경우/Noun', '가게/Noun', '사람/Noun', '때문/Noun',  '시스템/Noun', '썩/Noun', '인적/Noun', '직원/Noun',  '너/Noun', '캐치/Noun',  '재료/Noun', '청주/Noun', '온/Noun', '남자친구/Noun', '기분/Noun', '가게/Noun', '초밥/Noun', '사람/Noun', '왜/Noun', '마스크/Noun', '하나/Noun', '안/Noun', '끼/Noun', '조리/Noun', '사용/Noun', '입/Noun', '가리개/Noun', '코로나/Noun', '생각/Noun', '도/Noun', '안/Noun', '하나/Noun', '여자/Noun', '둘이서/Noun', '연어/Noun', '광어/Noun', '둘/Noun', '편이/Noun', '초밥/Noun', '연어/Noun', '광어/Noun', '번갈아/Noun', '가면/Noun', '주문/Noun', '주문/Noun', '때/Noun', '자꾸/Noun', '더/Noun', '주문/Noun', '건/Noun', '거기/Noun', '이해/Noun', '광어/Noun', '먼저/Noun', '판/Noun', '주더/Noun', '점/Noun', '정도/Noun', '몇번/Noun', '테이블/Noun', '연어/Noun', '연어/Noun', '회/Noun', '상태/Noun', '제각각/Noun', '테이블/Noun', '간격/Noun', '거기/Noun', '밥/Noun', '제멋대로/Noun', '왜/Noun', '인기/Noun', '코스/Noun', '요리/Noun', '타이밍/Noun', '고려/Noun', '음식/Noun', '모로/Noun', '재/Noun', '방문/Noun', '의사/Noun', '곳/Noun', '창원/Noun', '스시/Noun', '집임/Noun', '단점/Noun', '테이블/Noun', '간격/Noun', '줄/Noun', '기니/Noun', '예약/Noun', '셈/Noun',  '시럽/Noun', '축구/Noun', '경기/Noun', '창원/Noun', '지인/Noun',  '처음/Noun', '생맥주/Noun', '선택/Noun', '가로수길/Noun', '업체/Noun', '주류/Noun', '판매/Noun', '금지/Noun', '이/Noun', '밥/Noun', '정말/Noun', '예술/Noun', '처음/Noun', '젓가락/Noun', '것/Noun', '처럼/Noun', '끝내/Noun', '입/Noun', '입/Noun', '안/Noun', '밥알/Noun', '부/Noun', '미리/Noun', '예약/Noun', '시/Noun', '저녁/Noun', '국물/Noun',  '면/Noun', '불어/Noun',  '국물/Noun', '나이스/Noun', '함/Noun', '면/Noun', '하마/Noun', '츠/Noun', '이치란/Noun', '면/Noun', '종류/Noun', '추가/Noun', '더/Noun', '것/Noun', '테이블/Noun', '회전/Noun', '빨/Noun', '대기/Noun', '순서도/Noun',  '곳도/Noun', '무엇/Noun', '제일/Noun', '나중/Noun', '또/Noun', '가모/Noun', '매력/Noun', '자주/Noun', '일본/Noun', '라멘/Noun', '국내외/Noun', '군데/Noun', '여기/Noun', '제/Noun', '입맛/Noun', '완전/Noun', '지식/Noun', '먹기/Noun',   '한국/Noun', '거/Noun', '이/Noun', '지역/Noun', '사람/Noun', '볼일/Noun', '최근/Noun', '두/Noun', '번/Noun', '일회용/Noun', '컵/Noun',  '별/Noun', '하나/Noun', '후쿠오카/Noun', '번/Noun', '정도/Noun', '방문/Noun', '라멘/Noun', '파이터/Noun', '곳/Noun', '한국/Noun', '라면/Noun', '집/Noun', '라멘/Noun', '가대/Noun', '한국/Noun', '라면/Noun', '생각/Noun', '하이/Noun', '볼/Noun', '제일/Noun', '시국/Noun', '정도/Noun', '라멘/Noun', '집/Noun', '교자/Noun', '닭튀김/Noun', '번은/Noun', '저/Noun', '항상/Noun', '바질/Noun', '라멘/Noun', '차슈/Noun', '계란/Noun', '추가/Noun', '완전/Noun',  '리뷰/Noun', '지금/Noun', '침/Noun', '젓가락/Noun', 
      '숟가락/Noun', '용기/Noun', '등/Noun', '식기/Noun', '플라스틱/Noun',  '개념/Noun', '듯/Noun', '닭튀김/Noun', '사이드/Noun', '개창/Noun', '렬임/Noun', '차슈/Noun', '맛남/Noun', '생각/Noun', '바질/Noun', '맨/Noun', '라멘/Noun', '더/Noun', '이/Noun',  '이/Noun', '정도/Noun', '라멘/Noun', '나/Noun', '친구/Noun', '커플/Noun', '남자/Noun', '둘/Noun', '사람/Noun', '우리/Noun', '국물/Noun',  '달/Noun', '꽤/Noun', '바질/Noun', '분/Noun', '바질/Noun', '라멘/Noun', '꼭/Noun', '다가/Noun', '여기/Noun', '배민/Noun', '진해/Noun', '라면/Noun', '배달/Noun', '곳/Noun', '여기/Noun', '배달/Noun', '플로/Noun', '진해/Noun', '진짜/Noun', '달걀/Noun',  '라면/Noun', '차슈/Noun', '입구/Noun', '쪽/Noun', '또/Noun', '가야/Noun', '징/Noun', '커피/Noun', '빵/Noun',  '쇼케이스/Noun', '안/Noun', '빵류/Noun',  '과일/Noun', '몽블랑/Noun', '생크림/Noun', '언제/Noun', '눅눅/Noun', '자체/Noun', '핸드/Noun', '드립/Noun', '요/Noun', '개맛/Noun', '직원/Noun', '저번/Noun', '기억/Noun', '다시/Noun', '주차장/Noun', '관리인/Noun', '말귀/Noun', '이야기/Noun', '번/Noun', '그/Noun', '말/Noun', '바로/Noun', '핸들/Noun', '다른/Noun', '곳/Noun', '저/Noun', '리뷰/Noun', '안/Noun', '저녁/Noun', '때/Noun', '글/Noun', '주차장/Noun', '분/Noun', '관리/Noun',  '곳/Noun', '갈바/Noun', '집/Noun', '똥/Noun', '이모/Noun', '편/Noun', '집/Noun', '번은/Noun', '별/Noun', '점/Noun', '자체/Noun', '의미/Noun', '옴/Noun', '음식/Noun', '자기/Noun', '문안/Noun', '말/Noun', '국박/Noun', '손님/Noun', '식당/Noun',  '식당/Noun', '리뷰/Noun', '평생/Noun', '처음/Noun', '시간/Noun', '손님/Noun',  '종업원/Noun', '나이/Noun', '부모님/Noun', '참고/Noun', '계산/Noun', '때/Noun', '카운터/Noun', '마음/Noun', '얘기/Noun', '대신/Noun', '사과/Noun', '번/Noun', '다시/Noun', '분지/Noun', '음식/Noun', '직원/Noun', '언제/Noun', '이야기/Noun', '안감/Noun', '함안/Noun', '대구/Noun', '식당/Noun', '도청/Noun', '앞/Noun', '양각/Noun', '사파동/Noun', '수정/Noun', '골/Noun', '북면/Noun', '진가/Noun', '등등/Noun', '중/Noun', '가장/Noun', '유명/Noun', '세/Noun', '메뉴판/Noun', '젤위/Noun', '가장/Noun', '단가/Noun', '석쇠/Noun', '불고기/Noun', '장사/Noun', '급/Noun', '모습/Noun', '국밥/Noun', '물김치/Noun', '반찬/Noun', '깍두기/Noun', '생/Noun', '마늘/Noun', '손절각/Noun', '만/Noun', '옛날/Noun', '국밥/Noun', '사실/Noun', '루/Noun', '불고기/Noun', '정말/Noun', '과거/Noun', '영광/Noun', '백김치/Noun', '깍두기/Noun', '판매/Noun', '박일/Noun', '정도/Noun', '요/Noun', '쌈/Noun', '된장/Noun', '통/Noun', '퍼/Noun', '직접/Noun', '뒤론/Noun', '절대/Noun', '안감/Noun', '추억/Noun', '고기/Noun', '맛잏는데/Noun', '국/Noun', '음식/Noun', '조합/Noun',  '뭔가/Noun', '불고기/Noun',  '대비/Noun', '보통/Noun', '소고기/Noun', '국밥/Noun', '땡/Noun',  '조음/Noun', '샐러드/Noun',  '맛잇엇어/Noun', '하오츠/Noun', '하오츠/Noun',  '정돈/Noun', '정말/Noun',  '쵝오네/Noun', '해물/Noun', '베이스/Noun', '떡볶이/Noun', '샐러드/Noun', '이집/Noun', '데이트/Noun', '코스/Noun',  '크림/Noun', '파스타/Noun', '식기/Noun', '전/Noun', '진짜/Noun', '듯/Noun', '샐러드/Noun', '안/Noun', '편/Noun', '샐러드/Noun', '드레싱/Noun', '새우/Noun', '닭/Noun', '가슴/Noun', '살/Noun', '치즈/Noun', '떡볶이/Noun', '역시/Noun', '청춘/Noun', '키친/Noun', '답/Noun', '직원/Noun', '최/Noun', '샐러드/Noun',  '용/Noun', '양도/Noun', '성비/Noun',  '이용/Noun',  '샐러드/Noun', '떡볶이/Noun', '모두/Noun', '때/Noun', '항상/Noun',   '어유/Noun',  '아침/Noun', '샐러드/Noun', '지도/Noun', '여기/Noun', '샐러드/Noun', '꼭/Noun', '진짜/Noun', '집/Noun', '대기/Noun', '샐러드/Noun', '청춘/Noun', '치떡/Noun', '꼭/Noun', '함/Noun', '년/Noun', '전/Noun', '군대/Noun', '후임/Noun', '기억/Noun', '이제야/Noun', '리뷰/Noun',  '부산/Noun', '다음/Noun', '창원/Noun', '일/Noun', '꼭/Noun', '구/Noun', '비임/Noun',  '진짜/Noun', '샐러드/Noun', '꼭/Noun', '무슨/Noun', '풀/Noun', '양도/Noun', '대박/Noun', '내/Noun', '샐러드/Noun', '점/Noun',  '떡볶이/Noun', '맥주/Noun', '당/Noun', '스파게티/Noun', '맛남/Noun', '줄/Noun', '손님/Noun',  '대비/Noun', '양도/Noun', '정말/Noun', '바람/Noun', '직원/Noun', '얼굴/Noun', '음료/Noun',  '주시/Noun', '지금/Noun', '로이/Noun', '내/Noun', '파스타/Noun', '제일/Noun', '완전/Noun',  '창원/Noun', '꼭/Noun', '곳/Noun',  '치떡/Noun', '절대/Noun', '마/Noun', '응대/Noun', '예전/Noun', '사장/Noun', '언니/Noun', '동생/Noun', '손님/Noun', '곳/Noun', '건물/Noun', '축하/Noun', '명/Noun',  '룸/Noun', '예약/Noun', '해/Noun', '두상/Noun', '주문/Noun', '해도/Noun', '돈/Noun', '손해/Noun', '볼/Noun', '수/Noun', '배/Noun', '말/Noun', '절로/Noun', '옛날/Noun', '영도/Noun', '인심/Noun', '옛날/Noun', '영도/Noun', '옛날/Noun', '도장/Noun',   '비주/Noun', '얼/Noun', '또한/Noun', '빵점/Noun',  '서울/Noun', '손님/Noun', '모시/Noun', '룸/Noun', '안/Noun',  '정말/Noun', '이/Noun', '대자/Noun', '소자/Noun', '구분/Noun', '예전/Noun', '요즘/Noun', '장사/Noun',  '줄/Noun', '난리/Noun', '임/Noun', '성인/Noun', '인/Noun', '대자/Noun', '개시/Noun', '키/Noun', '아마/Noun', '장어/Noun', '무난/Noun',  '장어/Noun', '탕/Noun', '공기밥/Noun', '주차/Noun',  '정말/Noun', '휴가/Noun', '부모님/Noun', '처음/Noun',  '구/Noun', '응대/Noun', '것/Noun',  '양도/Noun',  '왜/Noun', '리뷰/Noun', '작성/Noun', '기분/Noun', '상해/Noun', '카카오/Noun', '맵/Noun', '가입/Noun', '리뷰/Noun', '제피/Noun', '방아/Noun', '장어/Noun', '국/Noun', '생/Noun', '생강/Noun', '슬/Noun', '라이스/Noun', '절/Noun', '생강/Noun', '붕장어/Noun', '구이/Noun',  '인테리어/Noun', '넘흐나/Noun', '커피/Noun', '실/Noun', '타래/Noun', '빙수/Noun', '직원/Noun', '강아지/Noun', '해/Noun', '주심/Noun', '맛모아욤/Noun', '내부/Noun', '볼거리/Noun', '곳/Noun', '힐링/Noun', '레이스/Noun', '식물/Noun', '카페/Noun', '마련/Noun', '식물/Noun', '잔뜩/Noun', '좌석/Noun', '테이블/Noun', '시선/Noun', '차단/Noun', '사진/Noun', '식물/Noun', '차/Noun', '한잔/Noun', '나무/Noun', '색/Noun', '다만/Noun', '커피/Noun',   '그/Noun',  '용서/Noun', '국적/Noun',  '곳/Noun', '인/Noun', '음료/Noun', '빙수/Noun', '인/Noun', '줌/Noun', '물/Noun', '얼음/Noun', '녹차/Noun', '베이스/Noun', '부어/Noun', '준/Noun', '옛날/Noun', '빙수/Noun', '느낌/Noun', '아침/Noun', '일찍/Noun', '사람/Noun', '가족/Noun', '시간/Noun', '옴/Noun', '바다/Noun',   '와플/Noun', '진심/Noun', '년전/Noun', '처음/Noun', '사람/Noun', '발/Noun', '지중해/Noun', '어제/Noun', '멀리/Noun', '온/Noun', '친구/Noun', '델꼬/Noun', '눈치/Noun', '역시/Noun', '저/Noun', '직접/Noun', '서빙/Noun', '분도/Noun', '세/Noun',  '출입/Noun', '금지/Noun', '경로당/Noun', '느낌/Noun', '그/Noun', '옛날/Noun', '지중해/Noun', '향수/Noun', '외각/Noun', '국적/Noun', '기/Noun', '요게/Noun', '별개/Noun', '준/Noun', '이유/Noun', '모든/Noun', '음료/Noun', '외면/Noun', '것/Noun', '치중/Noun', '도/Noun', '별개/Noun', '음/Noun', '컵/Noun', '립스틱/Noun', '자국/Noun', '그대로/Noun', '방금/Noun', '점/Noun', '메/Noun', '원/Noun', '드립/Noun', '커피/Noun', '카누/Noun', '와플/Noun', '아이스크림/Noun', '아이스크림/Noun', '컵/Noun', '바닥/Noun', '컵/Noun', '높이/Noun', '절반/Noun', '요/Noun',  '시체/Noun', '거의/Noun', '매운탕/Noun', '집/Noun', '바닷가/Noun', '근처/Noun', '와플/Noun', '이나/Noun', '그/Noun', '음료/Noun', '반드시/Noun', '사이드/Noun', '메뉴/Noun', '주문/Noun', '테이블/Noun', '순환/Noun', '때문/Noun', '겟/Noun', '메뉴/Noun', '몇개/Noun', '그것/Noun', '또/Noun', '그/Noun', '와중/Noun', '메뉴/Noun', '개/Noun', '시간/Noun', '가까이/Noun', '말/Noun', '한마디/Noun', '평범/Noun', '대기/Noun', '김/Noun',  '뭐/Noun', '한번/Noun', '볼/Noun', '하우스/Noun', '와인/Noun', '편의점/Noun', '원/Noun', '와인/Noun',  '보통/Noun',  '대기/Noun',  '주변/Noun', '이집/Noun', '사람/Noun', '제/Noun', '입맛/Noun', '개/Noun', '서울/Noun', '여기/Noun', '밥/Noun', '다시/Noun', '왜냐면/Noun', '진짜/Noun', '닭/Noun', '튀김/Noun', '국물/Noun', '고기랑/Noun', '진짜/Noun', '담/Noun', '그/Noun', '자체/Noun', '서울/Noun', '진심/Noun', '송파구/Noun', '쪽/Noun', '통닭/Noun', '다리/Noun', '쫄깃/Noun', '면발/Noun', '부모님/Noun', '모시/Noun', '별/Noun', '왜/Noun', '개/Noun',  '집/Noun', '닭다리/Noun', '하나/Noun', '다가/Noun', '가스/Noun', '우동/Noun', '이/Noun', '대기/Noun', '빠/Noun', '샐러드/Noun', '우동/Noun', '튀김/Noun', '인지/Noun', '안쪽/Noun', '튀김/Noun', '부분/Noun', '어쨋/Noun', '우동/Noun', '닭다리/Noun', '완전/Noun', '우동/Noun', '닭다리/Noun', '튀김/Noun', '배/Noun', '면발/Noun',  '쫄깃해/Noun', '국물/Noun', '시원/Noun', '닭튀김/Noun', '궁합/Noun', '다만/Noun', '닭다리/Noun', '냉동/Noun', '살이/Noun',   '별하나/Noun', '주말/Noun', '대기/Noun', '막/Noun',  '동네/Noun', '우동/Noun', '집/Noun', '생각/Noun', '가세/Noun',   '약간/Noun', '더/Noun', '닭다리/Noun', '하나/Noun', '여기/Noun', '감/Noun', '식/Noun', '자재/Noun', '마트/Noun', '아침/Noun', '쯤애/Noun', '것/Noun',   '만원/Noun', '조오오오오/Noun', '구리/Noun', '개인/Noun', '일본/Noun', '국물/Noun',  '치킨/Noun', '우동/Noun', '진짜/Noun',  '우동/Noun', '생각/Noun', '우동/Noun', '변경/Noun', '가능/Noun', '리코/Noun', '치즈/Noun', '빠/Noun', '샐러드/Noun', '하우스/Noun', '인도/Noun', '샐러드/Noun', '우동/Noun', '닭다리/Noun', '튀김/Noun', '바삭/Noun', '치킨/Noun', '하우스/Noun', '와인/Noun', '스/Noun', '위트/Noun', '지도/Noun', '드라이/Noun', '그/Noun', '중간/Noun', '진짜/Noun', '레몬/Noun', '스프레이/Noun', '꽃/Noun', '레몬/Noun', '즙/Noun', '뿌리/Noun', '것/Noun', '인상/Noun',  '또/Noun', '우동/Noun', '튀김/Noun', '닭/Noun', '매번/Noun', '오늘/Noun', '빵/Noun', '개/Noun', '때/Noun', '방문/Noun', '곳/Noun', '재/Noun', '방문/Noun', '사퍼/Noun', '시국/Noun', '주방/Noun', '턱스/Noun', '마스크/Noun', '기침/Noun', '빵/Noun', '빵/Noun', '밀크쉐이크/Noun', '명성/Noun', '처빵/Noun', '점원/Noun', '옆/Noun', '정신/Noun', '종류/Noun', '하나/Noun', '하나/Noun', '정말/Noun', '빵/Noun', '여름철/Noun', '창동/Noun', '김/Noun', '밀크쉐이크/Noun', '하나/Noun', '빵집/Noun', '그/Noun',  '그/Noun',  '바게트/Noun', '빵/Noun',  '꿀빵/Noun',  '구매/Noun', '빵/Noun', '영업/Noun', '분/Noun', '말솜씨/Noun', '역시/Noun', '빵/Noun', '코/Noun', '아보/Noun', '팥/Noun', '모든/Noun', '디저트/Noun', '빵/Noun', '매니아/Noun', '라면/Noun', '부근/Noun', '가면/Noun', '꼭/Noun', '곳/Noun', '옛날/Noun', '입맛/Noun', '전통/Noun', '단팥빵/Noun', '종류/Noun', '이성당/Noun', '전체/Noun',   '불친절/Noun', '기본/Noun']

In [20]:
search2 = "/Josa"
for i in range(244):
    for word in get_pos(df['ko_text'].values[i]):
        if search2 in word: 
            kk.append(word)  
stop_words = kk

In [21]:
search3 = "/Suffix" #접미사
for i in range(244):
    for word in get_pos(df['ko_text'].values[i]):
        if search3 in word: 
            kk.append(word)  
stop_words = kk

In [22]:
# stop_words = ['회맛/Noun', '나가야/Noun', '스끼/Noun', '회/Noun', '매운탕/Noun', '나오니/Noun', '마늘/Noun', '청양고추/Noun', '매운탕/Noun', '회/Noun', '매운탕/Noun', '회도/Noun', '매운탕/Noun', '지난번/Noun', '회/Noun', '회/Noun', '조개/Noun',  '공기밥/Noun', '횟집/Noun', '회맛/Noun', '물회/Noun', '장어/Noun', '탕/Noun', '물회/Noun', '듬회꼬/Noun', '치/Noun', '물회/Noun', '대게/Noun', '게/Noun', '밥/Noun', '점/Noun', '알/Noun', '차/Noun', '횟집/Noun', '회/Noun', '해산물/Noun', '게/Noun', '수족관/Noun', '것/Noun', '완판되는/Noun', '해산물/Noun', '횟집/Noun', '부산/Noun', '전국/Noun', '두루/Noun', '두루/Noun', '부산/Noun', '부산/Noun', '진해/Noun', '진해/Noun', '횟집/Noun', '부회/Noun', '타/Noun', '석화/Noun', '천원/Noun', '만원/Noun', '집앞/Noun', '횟집/Noun', '접수/Noun', '김치/Noun', '중국산/Noun', '밑반찬/Noun', '음식/Noun', '곳/Noun', '가게/Noun', '고기/Noun', '갈비/Noun', '오늘/Noun', '처음/Noun', '것/Noun', '창원/Noun', '손님/Noun', '모시/Noun', '집/Noun', '어디가/Noun', '곳/Noun', '블로그/Noun', '홍보/Noun', '글/Noun', '보고/Noun', '갈비/Noun', '마블링/Noun', '덩어리/Noun', '기름/Noun', '등심/Noun',  '향/Noun', '부위/Noun', '제공/Noun', '게다가/Noun', '냉면/Noun', '면/Noun', '면/Noun', '여행/Noun', '곳/Noun', '가요/Noun', '여행/Noun', '창원/Noun', '집/Noun', '재/Noun', '방문/Noun', '곳/Noun', '어른/Noun', '셋/Noun', '이서/Noun', '우모/Noun', '듬/Noun', '정식/Noun', '인분/Noun', '돼지갈비/Noun', '인분/Noun', '배/Noun', '여기/Noun', '반찬/Noun', '갈비/Noun', '이모/Noun', '오늘/Noun', '고기/Noun', '콜/Noun', '이모/Noun', '가지/Noun', '고기/Noun', '밑반찬/Noun', '짐/Noun', '점심/Noun', '한정식/Noun', '가게/Noun', '규모/Noun', '접객/Noun', '시스템/Noun', '매번/Noun', '휴가/Noun', '때/Noun', '가격/Noun', '점심/Noun',  '손님/Noun', '곳/Noun', '느낌/Noun', '제대로/Noun', '코로나/Noun', '안전/Noun', '및/Noun', '예방/Noun', '조치/Noun', '탁월/Noun', '보이/Noun', '모든/Noun', '서비스/Noun',  '제/Noun', '값/Noun', '함/Noun', '음식/Noun', '장소/Noun', '서비스/Noun', '가격/Noun',  '보장/Noun', '서브/Noun',  '직원/Noun', '밑반찬/Noun', '티/Noun', '고기/Noun', '제일/Noun', '일단/Noun', '벚꽃/Noun', '갈비/Noun', '양념/Noun', '이동/Noun', '갈비/Noun', '집중/Noun', '제일/Noun', '생갈/Noun', '비도/Noun', '소/Noun', '살살/Noun', '식감/Noun', '예전/Noun', '회식/Noun', '자주/Noun', '코로나/Noun', '담번/Noun', '우모/Noun', '듬/Noun', '가격/Noun', '옛날/Noun', '점심/Noun', '지금/Noun', '가격/Noun', '후/Noun', '덜덜/Noun', '자주/Noun', '상남동/Noun', '손님/Noun', '접대/Noun', '때/Noun', '꼭/Noun', '곳/Noun',  '곳/Noun', '강/Noun', '로타리/Noun', '클럽/Noun', '행사/Noun', '층/Noun', '진행/Noun', '수/Noun', '음식/Noun', '달/Noun', '종류/Noun', '젓가락/Noun', '곳/Noun', '상남동/Noun', '자리/Noun', '값/Noun', '이건/Noun', '평가/Noun', '가격/Noun', '듯/Noun', '음식/Noun', '전부/Noun', '달기/Noun', '특색/Noun', '광고/Noun', '완전/Noun', '점심/Noun', '가격/Noun', '것/Noun', '서비스/Noun', '다시/Noun', '가격/Noun', '명가/Noun', '수준/Noun', '서비스/Noun', '일반/Noun',  '수준/Noun', '임/Noun', '안심/Noun', '가격/Noun', '아이/Noun', '이집/Noun', '육회/Noun', '매우/Noun', '냉면/Noun', '평양/Noun', '냉면/Noun', '진주/Noun', '하연/Noun', '옥/Noun', '냉면/Noun', '식/Noun', '창원/Noun', '냉면/Noun', '개인/Noun', '가장/Noun', '취향/Noun', '원/Noun', '정식/Noun', '엄마/Noun', '할머니/Noun', '모시/Noun', '두/Noun', '분/Noun',  '괜찬음/Noun', '이/Noun', '가격/Noun', '서비스/Noun', '친구/Noun', '고기/Noun', '먼저/Noun', '양념/Noun', '소갈비/Noun', '소갈비/Noun', '무엇/Noun', '반찬/Noun', '서비스/Noun', '분/Noun', '다음/Noun', '번/Noun', '가족/Noun', '모임/Noun', '여기/Noun', '할머님/Noun', '듯/Noun', '참고/Noun', '불도/Noun', '숯불/Noun', '오늘/Noun',  '갈비/Noun', '가격/Noun', '대비/Noun', '서비스/Noun', '쫌/Noun', '커피/Noun', '티라미수/Noun', '면/Noun', '바싹/Noun',  '곳/Noun',  '티라미수/Noun', '상태/Noun', '뮤/Noun', '상한/Noun', '거/Noun', '얘기/Noun', '안/Noun', '배/Noun', '사장/Noun', '리뷰/Noun', '보시/Noun', '조심/Noun',  '가격/Noun', '뷰/Noun', '딸/Noun', '최애/Noun', '레이스/Noun', '레인보우/Noun', '케잌/Noun', '곳/Noun', '나머진/Noun', '중간/Noun', '정도/Noun', '커피/Noun', '경남/Noun', '엔젤리너스/Noun', '맹탕/Noun', '중간/Noun', '이상은/Noun',  '브라운필드/Noun', '리노/Noun', '베이/Noun', '션/Noun', '브라운/Noun', '핸즈/Noun', '값/Noun',  '맥주/Noun', '팔고/Noun', '연/Noun', '옴/Noun', '염장/Noun', '주의/Noun', '욘/Noun', '뷰/Noun', '오점/Noun',  '바람/Noun', '카페/Noun', '라떼/Noun', '천원/Noun', '커피/Noun', '낮/Noun', '가면/Noun', '뷰/Noun', '뷰/Noun', '커피/Noun', '테이블/Noun', '다리/Noun', '높이/Noun', '계속/Noun', '임/Noun', '점원/Noun', '소통/Noun', '서비스/Noun',  '야경/Noun', '뷰/Noun', '정말/Noun', '예쁘댜/Noun', '음료/Noun', '빙수/Noun', '가격/Noun', '반의/Noun', '반도/Noun', '매우/Noun', '가격/Noun', '라떼/Noun', '아이스/Noun', '라떼/Noun', '타코/Noun', '내부/Noun', '생각/Noun', '메뉴/Noun', '창원/Noun', '집/Noun', '타코/Noun', '종류/Noun', '메뉴/Noun', '더/Noun', '타코/Noun', '가격/Noun', '대비/Noun', '생각/Noun', '가격/Noun', '괜춘/Noun', '무한/Noun', '나쵸/Noun', '그거/Noun', '배/Noun', '먹음/Noun', '서울/Noun', '자주/Noun', '타코/Noun', '집들이/Noun', '비교/Noun', '때/Noun', '정말/Noun',  '가격/Noun', '서비스/Noun', '불친절/Noun', '서비스/Noun',  '평범/Noun', '돈/Noun', '안감/Noun', '양도/Noun', '은근/Noun', '사람/Noun',   '독보/Noun', '사람/Noun', '타코/Noun', '타코/Noun', '념념굿/Noun', '생각/Noun', '음식/Noun', '망고/Noun', '타코/Noun', '비프/Noun', '퀘사디아/Noun', '쉬/Noun', '림프/Noun', '중/Noun', '갑/Noun', '망고/Noun', '타코/Noun', '규/Noun', '스테이크/Noun', '위/Noun', '시킴/Noun', '비프/Noun', '타코/Noun', '제일/Noun', '재/Noun', '방문/Noun', '의사/Noun', '프로/Noun', '타코/Noun', '재/Noun', '방문/Noun', '의사/Noun', '토/Noun', '도스/Noun', '메뉴/Noun', '란/Noun', '창원/Noun', '멕시코/Noun', '음식/Noun', '다만/Noun', '향신료/Noun',  '버섯/Noun', '튀김/Noun', '대안/Noun', '주관/Noun', '제/Noun', '입맛/Noun', '이태원/Noun', '멕시코/Noun', '식당/Noun', '더/Noun', '망고/Noun', '쉬/Noun', '림프/Noun', '타코/Noun', '조금/Noun', '느낌/Noun', '핫소스/Noun', '규/Noun', '스테이크/Noun', '진짜/Noun', '라이스/Noun', '전체/Noun', '만족/Noun', '토핑/Noun', '타코/Noun', '라이스/Noun', '치폴레/Noun', '조금/Noun', '양도/Noun', '완전/Noun', '만족/Noun', '방역/Noun', '관리/Noun', '제대로/Noun', '정말/Noun', '명부/Noun', '마스크/Noun', '착용/Noun', '셀프/Noun', '바/Noun', '마스크/Noun', '착용/Noun', '요청/Noun', '손소독/Noun', '후/Noun', '입장/Noun', '등/Noun', '에피타이저/Noun', '나초/Noun', '소스/Noun', '토/Noun', '도스/Noun', '그릴/Noun', '드/Noun', '파히타/Noun', '자체/Noun', '또띠아/Noun', '진심/Noun', '배/Noun', '또띠아/Noun', '리필/Noun', '가능/Noun', '가격/Noun', '조금/Noun', '것/Noun', '완벽/Noun', '창원/Noun', '곳/Noun', '멕시칸/Noun', '음식/Noun', '쫀맛입/Noun', '니드/Noun', '음식/Noun', '자주/Noun', '가요/Noun', '타코/Noun', '정통/Noun', '타코/Noun', '한국인/Noun', '입맛/Noun', '타코/Noun', '저/Noun', '더/Noun', '재료/Noun', '음식/Noun', '양/Noun', '등/Noun', '약간/Noun', '편/Noun', '듯/Noun', '타코/Noun', '고기/Noun', '때/Noun', '손님/Noun', '새우/Noun', '고기/Noun', '식/Noun', '만족/Noun', '고기/Noun',  '나/Noun', '편/Noun', '가격/Noun', '타코/Noun', '감자/Noun', '스프/Noun', '라임/Noun', '모히또/Noun', '꽤/Noun', '동굴/Noun', '동굴/Noun', '내부/Noun', '나름/Noun', '정리/Noun', '팔/Noun', '겉옷/Noun',  '여름/Noun', '입김/Noun', '셀프/Noun', '바/Noun', '동굴/Noun', '미터/Noun', '벨/Noun', '종업원/Noun', '어려움/Noun', '동굴/Noun', '시체/Noun', '동굴/Noun',   '밑반찬/Noun', '샐러드/Noun', '셀프/Noun', '바/Noun', '홀/Noun', '호출/Noun', '버튼/Noun', '오리/Noun', '살짝/Noun', '남/Noun', '예약/Noun', '다해/Noun', '매장/Noun', '기표/Noun', '손님/Noun', '눈앞/Noun', '일단/Noun', '동굴/Noun', '것/Noun', '자체/Noun', '장점/Noun', '곳/Noun', '정말/Noun', '오리/Noun', '불고기/Noun', '오리탕/Noun', '오리/Noun', '불고기/Noun', '자극/Noun', '오리탕/Noun', '처음/Noun', '닭개장/Noun', '제/Noun', '입/Noun', '오리/Noun', '불고기/Noun', '더/Noun', '오리/Noun', '불고기/Noun', '백숙/Noun', '진짜/Noun',  '특색/Noun', '오리/Noun', '백숙/Noun',  '무엇/Noun',  '남자/Noun', '사장/Noun', '직원/Noun', '일/Noun', '분/Noun', '정말/Noun', '카운터/Noun', '남/Noun', '직원/Noun', '일/Noun', '진짜/Noun', '개애애못함/Noun',  '비도/Noun', '것/Noun', '다만/Noun', '제/Noun', '때/Noun', '워낙/Noun', '묵은지/Noun', '계산/Noun', '때/Noun', '묵은지/Noun', '묵은지/Noun', '찬/Noun', '서빙/Noun', '뭐/Noun', '표정/Noun', '말/Noun', '안/Noun',  '길/Noun',  '단점/Noun', '초밥/Noun', '포장/Noun', '예전/Noun', '포장/Noun', '집/Noun', '스시/Noun', '훨/Noun', '요/Noun', '천안/Noun', '상/Noun', '초밥/Noun', '것/Noun', '오늘/Noun', '초밥/Noun', '시반/Noun', '전/Noun', '예약/Noun', '시/Noun', '분/Noun', '감/Noun', '점심시간/Noun', '층/Noun', '분/Noun', '진짜/Noun',  '정식/Noun', '코스/Noun', '기억/Noun', '창원/Noun', '가면/Noun', '또/Noun', '애오/Noun',  '점점/Noun', '곳/Noun', '곳/Noun', '테이블/Noun', '간격/Noun', '코스/Noun', '서브/Noun', '순서/Noun', '인/Noun', '테이블/Noun', '사시미/Noun', '먹기/Noun', '전/Noun', '초밥/Noun', '서브/Noun', '그릇/Noun', '그릇/Noun', '전/Noun', '다음/Noun', '음식/Noun', '틈/Noun', '모서리/Noun', '경우/Noun', '가게/Noun', '사람/Noun', '때문/Noun',  '시스템/Noun', '썩/Noun', '인적/Noun', '직원/Noun', '실수/Noun', '너/Noun', '캐치/Noun', '서비스/Noun', '재료/Noun', '청주/Noun', '온/Noun', '남자친구/Noun', '기분/Noun', '가게/Noun', '초밥/Noun', '사람/Noun', '왜/Noun', '마스크/Noun', '하나/Noun', '안/Noun', '끼/Noun', '조리/Noun', '사용/Noun', '입/Noun', '가리개/Noun', '코로나/Noun', '생각/Noun', '도/Noun', '안/Noun', '하나/Noun', '여자/Noun', '둘이서/Noun', '연어/Noun', '광어/Noun', '둘/Noun', '편이/Noun', '초밥/Noun', '연어/Noun', '광어/Noun', '번갈아/Noun', '가면/Noun', '주문/Noun', '주문/Noun', '때/Noun', '자꾸/Noun', '더/Noun', '주문/Noun', '건/Noun', '거기/Noun', '이해/Noun', '광어/Noun', '먼저/Noun', '판/Noun', '주더/Noun', '점/Noun', '정도/Noun', '몇번/Noun', '테이블/Noun', '연어/Noun', '연어/Noun', '회/Noun', '상태/Noun', '제각각/Noun', '테이블/Noun', '간격/Noun', '거기/Noun', '밥/Noun', '제멋대로/Noun', '왜/Noun', '인기/Noun', '코스/Noun', '요리/Noun', '타이밍/Noun', '고려/Noun', '음식/Noun', '모로/Noun', '재/Noun', '방문/Noun', '의사/Noun', '곳/Noun', '창원/Noun', '스시/Noun', '집임/Noun', '단점/Noun', '테이블/Noun', '간격/Noun', '줄/Noun', '기니/Noun', '예약/Noun', '셈/Noun',  '시럽/Noun', '축구/Noun', '경기/Noun', '창원/Noun', '지인/Noun',  '처음/Noun', '생맥주/Noun', '선택/Noun', '가로수길/Noun', '업체/Noun', '주류/Noun', '판매/Noun', '금지/Noun', '이/Noun', '밥/Noun', '정말/Noun', '예술/Noun', '처음/Noun', '젓가락/Noun', '것/Noun', '처럼/Noun', '끝내/Noun', '입/Noun', '입/Noun', '안/Noun', '밥알/Noun', '부/Noun', '미리/Noun', '예약/Noun', '시/Noun', '저녁/Noun', '국물/Noun',  '면/Noun', '불어/Noun',  '국물/Noun', '나이스/Noun', '함/Noun', '면/Noun', '하마/Noun', '츠/Noun', '이치란/Noun', '면/Noun', '종류/Noun', '추가/Noun', '더/Noun', '것/Noun', '테이블/Noun', '회전/Noun', '빨/Noun', '대기/Noun', '순서도/Noun',  '곳도/Noun', '무엇/Noun', '제일/Noun', '나중/Noun', '또/Noun', '가모/Noun', '매력/Noun', '자주/Noun', '일본/Noun', '라멘/Noun', '국내외/Noun', '군데/Noun', '여기/Noun', '제/Noun', '입맛/Noun', '완전/Noun', '지식/Noun', '먹기/Noun',   '한국/Noun', '거/Noun', '이/Noun', '지역/Noun', '사람/Noun', '볼일/Noun', '최근/Noun', '두/Noun', '번/Noun', '일회용/Noun', '컵/Noun',  '별/Noun', '하나/Noun', '후쿠오카/Noun', '번/Noun', '정도/Noun', '방문/Noun', '라멘/Noun', '파이터/Noun', '곳/Noun', '한국/Noun', '라면/Noun', '집/Noun', '라멘/Noun', '가대/Noun', '한국/Noun', '라면/Noun', '생각/Noun', '하이/Noun', '볼/Noun', '제일/Noun', '시국/Noun', '정도/Noun', '라멘/Noun', '집/Noun', '교자/Noun', '닭튀김/Noun', '번은/Noun', '저/Noun', '항상/Noun', 
#               '바질/Noun', '라멘/Noun', '차슈/Noun', '계란/Noun', '추가/Noun', '완전/Noun',  '리뷰/Noun', '지금/Noun', '침/Noun', '젓가락/Noun', '숟가락/Noun', '용기/Noun', '등/Noun', '식기/Noun', '플라스틱/Noun',  '개념/Noun', '듯/Noun', '닭튀김/Noun', '사이드/Noun', '개창/Noun', '렬임/Noun', '차슈/Noun', '맛남/Noun', '생각/Noun', '바질/Noun', '맨/Noun', '라멘/Noun', '더/Noun', '이/Noun', '가격/Noun', '이/Noun', '정도/Noun', '라멘/Noun', '나/Noun', '친구/Noun', '커플/Noun', '남자/Noun', '둘/Noun', '사람/Noun', '우리/Noun', '국물/Noun',  '달/Noun', '꽤/Noun', '바질/Noun', '분/Noun', '바질/Noun', '라멘/Noun', '꼭/Noun', '다가/Noun', '여기/Noun', '배민/Noun', '진해/Noun', '라면/Noun', '배달/Noun', '곳/Noun', '여기/Noun', '배달/Noun', '플로/Noun', '진해/Noun', '진짜/Noun', '달걀/Noun',  '라면/Noun', '차슈/Noun', '입구/Noun', '쪽/Noun', '또/Noun', '가야/Noun', '징/Noun', '커피/Noun', '빵/Noun',  '쇼케이스/Noun', '안/Noun', '빵류/Noun',  '과일/Noun', '몽블랑/Noun', '생크림/Noun', '언제/Noun', '눅눅/Noun', '자체/Noun', '핸드/Noun', '드립/Noun', '요/Noun', '개맛/Noun', '직원/Noun', '저번/Noun', '기억/Noun', '다시/Noun', '주차장/Noun', '관리인/Noun', '말귀/Noun', '이야기/Noun', '번/Noun', '그/Noun', '말/Noun', '바로/Noun', '핸들/Noun', '다른/Noun', '곳/Noun', '저/Noun', '리뷰/Noun', '안/Noun', '저녁/Noun', '때/Noun', '글/Noun', '주차장/Noun', '분/Noun', '관리/Noun',  '곳/Noun', '갈바/Noun', '집/Noun', '똥/Noun', '이모/Noun', '편/Noun', '집/Noun', '번은/Noun', '별/Noun', '점/Noun', '자체/Noun', '의미/Noun', '옴/Noun', '음식/Noun', '자기/Noun', '문안/Noun', '말/Noun', '국박/Noun', '손님/Noun', '식당/Noun', '그닥/Noun', '식당/Noun', '리뷰/Noun', '평생/Noun', '처음/Noun', '시간/Noun', '손님/Noun',  '종업원/Noun', '나이/Noun', '부모님/Noun', '참고/Noun', '계산/Noun', '때/Noun', '카운터/Noun', '마음/Noun', '얘기/Noun', '대신/Noun', '사과/Noun', '번/Noun', '다시/Noun', '분지/Noun', '음식/Noun', '직원/Noun', '언제/Noun', '이야기/Noun', '안감/Noun', '함안/Noun', '대구/Noun', '식당/Noun', '도청/Noun', '앞/Noun', '양각/Noun', '사파동/Noun', '수정/Noun', '골/Noun', '북면/Noun', '진가/Noun', '등등/Noun', '중/Noun', '가장/Noun', '유명/Noun', '세/Noun', '메뉴판/Noun', '젤위/Noun', '가장/Noun', '단가/Noun', '석쇠/Noun', '불고기/Noun', '장사/Noun', '급/Noun', '모습/Noun', '국밥/Noun', '물김치/Noun', '반찬/Noun', '깍두기/Noun', '생/Noun', '마늘/Noun', '손절각/Noun', '만/Noun', '옛날/Noun', '국밥/Noun', '사실/Noun', '루/Noun', '불고기/Noun', '정말/Noun', '과거/Noun', '영광/Noun', '백김치/Noun', '깍두기/Noun', '판매/Noun', '박일/Noun', '정도/Noun', '요/Noun', '쌈/Noun', '된장/Noun', '통/Noun', '퍼/Noun', '직접/Noun', '뒤론/Noun', '절대/Noun', '안감/Noun', '추억/Noun', '고기/Noun', '맛잏는데/Noun', '국/Noun', '음식/Noun', '조합/Noun', '아쉬움/Noun', '뭔가/Noun', '불고기/Noun', '가격/Noun', '대비/Noun', '보통/Noun', '소고기/Noun', '국밥/Noun', '땡/Noun',  '조음/Noun', '샐러드/Noun', '강추/Noun', '맛잇엇어/Noun', '하오츠/Noun', '하오츠/Noun',  '정돈/Noun', '정말/Noun',  '쵝오네/Noun', '해물/Noun', '베이스/Noun', '떡볶이/Noun', '샐러드/Noun', '이집/Noun', '데이트/Noun', '코스/Noun',  '크림/Noun', '파스타/Noun', '식기/Noun', '전/Noun', '진짜/Noun', '듯/Noun', '샐러드/Noun', '안/Noun', '편/Noun', '샐러드/Noun', '드레싱/Noun', '새우/Noun', '닭/Noun', '가슴/Noun', '살/Noun', '치즈/Noun', '떡볶이/Noun', '역시/Noun', '청춘/Noun', '키친/Noun', '답/Noun', '직원/Noun', '최/Noun', '샐러드/Noun',  '용/Noun', '양도/Noun', '성비/Noun',  '이용/Noun',  '샐러드/Noun', '떡볶이/Noun', '모두/Noun', '때/Noun', '항상/Noun', '굿/Noun', '굿/Noun', '어유/Noun',  '아침/Noun', '샐러드/Noun', '지도/Noun', '여기/Noun', '샐러드/Noun', '꼭/Noun', '진짜/Noun', '집/Noun', '대기/Noun', '샐러드/Noun', '청춘/Noun', '치떡/Noun', '꼭/Noun', '함/Noun', '년/Noun', '전/Noun', '군대/Noun', '후임/Noun', '기억/Noun', '이제야/Noun', '리뷰/Noun',  '부산/Noun', '다음/Noun', '창원/Noun', '일/Noun', '꼭/Noun', '구/Noun', '비임/Noun', '그닥/Noun', '진짜/Noun', '샐러드/Noun', '꼭/Noun', '무슨/Noun', '풀/Noun', '양도/Noun', '대박/Noun', '내/Noun', '샐러드/Noun', '점/Noun',  '떡볶이/Noun', '맥주/Noun', '당/Noun', '스파게티/Noun', '맛남/Noun', '줄/Noun', '손님/Noun', '가격/Noun', '대비/Noun', '양도/Noun', '정말/Noun', '바람/Noun', '직원/Noun', '얼굴/Noun', '음료/Noun', '서비스/Noun', '주시/Noun', '지금/Noun', '로이/Noun', '내/Noun', '파스타/Noun', '제일/Noun', '완전/Noun',  '창원/Noun', '꼭/Noun', '곳/Noun', '가격/Noun', '치떡/Noun', '절대/Noun', '마/Noun', '응대/Noun', '예전/Noun', '사장/Noun', '언니/Noun', '동생/Noun', '손님/Noun', '곳/Noun', '건물/Noun', '축하/Noun', '명/Noun',  '룸/Noun', '예약/Noun', '해/Noun', '두상/Noun', '주문/Noun', '해도/Noun', '돈/Noun', '손해/Noun', '볼/Noun', '수/Noun', '배/Noun', '말/Noun', '절로/Noun', '옛날/Noun', '영도/Noun', '인심/Noun', '옛날/Noun', '영도/Noun', '옛날/Noun', '도장/Noun', '가격/Noun', '가격/Noun', '비주/Noun', '얼/Noun', '또한/Noun', '빵점/Noun',  '서울/Noun', '손님/Noun', '모시/Noun', '룸/Noun', '안/Noun', '양/Noun', '정말/Noun', '이/Noun', '대자/Noun', '소자/Noun', '구분/Noun', '예전/Noun', '요즘/Noun', '장사/Noun', '양/Noun', '줄/Noun', '난리/Noun', '임/Noun', '성인/Noun', '인/Noun', '대자/Noun', '개시/Noun', '키/Noun', '아마/Noun', '장어/Noun', '무난/Noun', '서비스/Noun', '장어/Noun', '탕/Noun', '공기밥/Noun', '주차/Noun',  '정말/Noun', '휴가/Noun', '부모님/Noun', '처음/Noun', '양/Noun', '구/Noun', '응대/Noun', '것/Noun',  '양도/Noun', '서비스/Noun', '왜/Noun', '리뷰/Noun', '작성/Noun', '기분/Noun', '상해/Noun', '카카오/Noun', '맵/Noun', '가입/Noun', '리뷰/Noun', '제피/Noun', '방아/Noun', '장어/Noun', '국/Noun', '생/Noun', '생강/Noun', '슬/Noun', '라이스/Noun', '절/Noun', '생강/Noun', '붕장어/Noun', '구이/Noun', '굿/Noun', '인테리어/Noun', '넘흐나/Noun', '커피/Noun', '실/Noun', '타래/Noun', '빙수/Noun', '직원/Noun', '강아지/Noun', '해/Noun', '주심/Noun', '맛모아욤/Noun', '내부/Noun', '볼거리/Noun', '곳/Noun', '힐링/Noun', '레이스/Noun', '식물/Noun', '카페/Noun', '마련/Noun', '식물/Noun', '잔뜩/Noun', '좌석/Noun', '테이블/Noun', '시선/Noun', '차단/Noun', '사진/Noun', '식물/Noun', '차/Noun', '한잔/Noun', '나무/Noun', '색/Noun', '다만/Noun', '커피/Noun', '가격/Noun',  '그/Noun', '가격/Noun', '용서/Noun', '국적/Noun',  '곳/Noun', '인/Noun', '음료/Noun', '빙수/Noun', '인/Noun', '줌/Noun', '물/Noun', '얼음/Noun', '녹차/Noun', '베이스/Noun', '부어/Noun', '준/Noun', '옛날/Noun', '빙수/Noun', '느낌/Noun', '아침/Noun', '일찍/Noun', '사람/Noun', '가족/Noun', '시간/Noun', '옴/Noun', '바다/Noun', '뷰/Noun', '뷰/Noun', '와플/Noun', '진심/Noun', '년전/Noun', '처음/Noun', '사람/Noun', '발/Noun', '지중해/Noun', '어제/Noun', '멀리/Noun', '온/Noun', '친구/Noun', '델꼬/Noun', '눈치/Noun', '역시/Noun', '저/Noun', '직접/Noun', '서빙/Noun', '분도/Noun', '세/Noun',  '출입/Noun', '금지/Noun', '경로당/Noun', '느낌/Noun', '그/Noun', '옛날/Noun', '지중해/Noun', '향수/Noun', '외각/Noun', '국적/Noun', '기/Noun', '요게/Noun', '별개/Noun', '준/Noun', '이유/Noun', '모든/Noun', '음료/Noun', '외면/Noun', '것/Noun', '치중/Noun', '도/Noun', '별개/Noun', '음/Noun', '컵/Noun', '립스틱/Noun', '자국/Noun', '그대로/Noun', '방금/Noun', '점/Noun', '메/Noun', '원/Noun', '드립/Noun', '커피/Noun', '카누/Noun', '와플/Noun', '아이스크림/Noun', '아이스크림/Noun', '컵/Noun', '바닥/Noun', '컵/Noun', '높이/Noun', '절반/Noun', '요/Noun', '뷰/Noun', '시체/Noun', '거의/Noun', '매운탕/Noun', '집/Noun', '바닷가/Noun', '근처/Noun', '와플/Noun', '이나/Noun', '그/Noun', '음료/Noun', '반드시/Noun', '사이드/Noun', '메뉴/Noun', '주문/Noun', '테이블/Noun', '순환/Noun', '때문/Noun', '겟/Noun', '메뉴/Noun', '몇개/Noun', '그것/Noun', '또/Noun', '그/Noun', '와중/Noun', '메뉴/Noun', '개/Noun', '시간/Noun', '가까이/Noun', '말/Noun', '한마디/Noun', '평범/Noun', '대기/Noun', '김/Noun', '임팩트/Noun', '뭐/Noun', '한번/Noun', '볼/Noun', '하우스/Noun', '와인/Noun', '편의점/Noun', '원/Noun', '와인/Noun',  '보통/Noun', '양/Noun', '대기/Noun',  '주변/Noun', '이집/Noun', '사람/Noun', '제/Noun', '입맛/Noun', '개/Noun', '서울/Noun', '여기/Noun', '밥/Noun', '다시/Noun', '왜냐면/Noun', '진짜/Noun', '닭/Noun', '튀김/Noun', '국물/Noun', '고기랑/Noun', '진짜/Noun', '담/Noun', '그/Noun', '자체/Noun', '서울/Noun', '진심/Noun', '송파구/Noun', '쪽/Noun', '통닭/Noun', '다리/Noun', '쫄깃/Noun', '면발/Noun', '부모님/Noun', '모시/Noun', '별/Noun', '왜/Noun', '개/Noun',  '집/Noun', '닭다리/Noun', '하나/Noun', '다가/Noun', '가스/Noun', '우동/Noun', '이/Noun', '대기/Noun', '빠/Noun', '샐러드/Noun', '우동/Noun', '튀김/Noun', '인지/Noun', '안쪽/Noun', '튀김/Noun', '부분/Noun', '어쨋/Noun', '우동/Noun', '닭다리/Noun', '완전/Noun', '우동/Noun', '닭다리/Noun', '튀김/Noun', '배/Noun', '면발/Noun', '살짝/Noun', '쫄깃해/Noun', '국물/Noun', '시원/Noun', '닭튀김/Noun', '궁합/Noun', '다만/Noun', '닭다리/Noun', '냉동/Noun', '살이/Noun',   '별하나/Noun', '주말/Noun', '대기/Noun', '막/Noun',  '동네/Noun', '우동/Noun', '집/Noun', '생각/Noun', '가세/Noun', '조금/Noun', '가격/Noun', '약간/Noun', '더/Noun', '닭다리/Noun', '하나/Noun', '여기/Noun', '감/Noun', '식/Noun', '자재/Noun', '마트/Noun', '아침/Noun', '쯤애/Noun', '것/Noun', '내년/Noun', '가격/Noun', '만원/Noun', '조오오오오/Noun', '구리/Noun', '개인/Noun', '일본/Noun', '국물/Noun',  '치킨/Noun', '우동/Noun', '진짜/Noun',  '우동/Noun', '생각/Noun', '우동/Noun', '변경/Noun', '가능/Noun', '리코/Noun', '치즈/Noun', '빠/Noun', '샐러드/Noun', '하우스/Noun', '인도/Noun', '샐러드/Noun', '우동/Noun', '닭다리/Noun', '튀김/Noun', '바삭/Noun', '치킨/Noun', '하우스/Noun', '와인/Noun', '스/Noun', '위트/Noun', '지도/Noun', '드라이/Noun', '그/Noun', '중간/Noun', '진짜/Noun', '레몬/Noun', '스프레이/Noun', '꽃/Noun', '레몬/Noun', '즙/Noun', '뿌리/Noun', '것/Noun', '인상/Noun',  '또/Noun', '우동/Noun', '튀김/Noun', '닭/Noun', '매번/Noun', '오늘/Noun', '빵/Noun', '개/Noun', '때/Noun', '방문/Noun', '곳/Noun', '재/Noun', '방문/Noun', '사퍼/Noun', '시국/Noun', '주방/Noun', '턱스/Noun', '마스크/Noun', '기침/Noun', '빵/Noun', '빵/Noun', '밀크쉐이크/Noun', '명성/Noun', '처빵/Noun', '점원/Noun', '옆/Noun', '정신/Noun', '종류/Noun', '하나/Noun', '하나/Noun', '정말/Noun', '빵/Noun', '여름철/Noun', '창동/Noun', '김/Noun', '밀크쉐이크/Noun', '하나/Noun', '빵집/Noun', '그/Noun',  '그/Noun',  '바게트/Noun', '빵/Noun',  '꿀빵/Noun',  '구매/Noun', '빵/Noun', '영업/Noun', '분/Noun', '말솜씨/Noun', '역시/Noun', '빵/Noun', '코/Noun', '아보/Noun', '팥/Noun', '모든/Noun', '디저트/Noun', '빵/Noun', '매니아/Noun', '라면/Noun', '부근/Noun', '가면/Noun', '꼭/Noun', '곳/Noun', '옛날/Noun', '입맛/Noun', '전통/Noun', '단팥빵/Noun', '종류/Noun', '이성당/Noun', '전체/Noun', '가격/Noun', '조금/Noun', '불친절/Noun', '기본/Noun']

In [55]:
print(len(stop_words))

2561


In [24]:
from sklearn.feature_extraction.text import CountVectorizer

# 형태소를 학습데이터셋으로 변환

index_vectorizer = CountVectorizer(tokenizer = lambda x: get_pos(x), stop_words=stop_words)#, stop_words=stop_words
X = index_vectorizer.fit_transform(df['ko_text'].tolist())
print(X.shape)

(244, 1055)


In [25]:
print(str(index_vectorizer.vocabulary_)[:100]+"..")

{'맛/Noun': 347, '난건/Noun': 174, '오전/Noun': 730, '품절/Noun': 977, '가성/Noun': 34, '비는/Verb': 511, '좋지만/..


In [26]:
print(str(index_vectorizer.vocabulary_))

{'맛/Noun': 347, '난건/Noun': 174, '오전/Noun': 730, '품절/Noun': 977, '가성/Noun': 34, '비는/Verb': 511, '좋지만/Adjective': 883, '너무/Adverb': 199, '시끄럽고/Adjective': 580, '없는/Adjective': 698, '모르고/Verb': 427, '후딱/Adverb': 1049, '먹고/Verb': 385, '빨리/Adverb': 527, '하는/Verb': 991, '그리고/Conjunction': 112, '하시는/Verb': 1001, '하신듯/Verb': 1005, '비/Noun': 510, '최고/Noun': 931, '소문/Noun': 569, '듣고/Verb': 281, '갔는데/Verb': 46, '필수/Noun': 979, '필요/Noun': 980, '없고/Adjective': 695, '저렴한/Adjective': 825, '가격/Noun': 16, '많이/Adverb': 343, '싶다/Verb': 610, '무조건/Noun': 435, '입니다/Adjective': 785, '삼삼하게/Adjective': 550, '기호/Noun': 132, '맞게/Verb': 373, '해서/Verb': 1025, '드세요/Verb': 278, '들/Verb': 283, '친절하시고/Adjective': 938, '모든/Modifier': 422, '매/Modifier': 381, '뉴/Noun': 213, '임니다/Verb': 781, '이/Determiner': 757, '성지/Noun': 563, '호불호/Noun': 1042, '갈립니다/Verb': 40, '저렴하고/Adjective': 824, '쾌적하고/Adjective': 948, '양/Noun': 680, '많고/Adjective': 335, '맛있어요/Adjective': 363, '주/Modifier': 886, '차요/Noun': 914, '불친절해요/Adjective': 502

In [27]:
from sklearn.feature_extraction.text import TfidfTransformer

# TF-IDF 방법
# 형태소를학습 데이터셋으로 변환
tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)

In [28]:
df['y'].value_counts()

1    163
0     81
Name: y, dtype: int64

In [29]:
df = df.reset_index(drop = True, inplace = False)

In [30]:
X.shape

(244, 1055)

In [31]:
# y.shape #y is not defined

In [32]:
from sklearn.model_selection import train_test_split
import random
# positive_random_idx = df[df['y'] == 1].sample(163, random_state = 10, replace=True).index.tolist()
# negative_random_idx = df[df['y'] == 0].sample(81, random_state = 10, replace=True).index.tolist()


# random_idx = positive_random_idx + negative_random_idx
# print(random_idx)
# sample_X = X[random_idx, :]
y = df['y'] # [random_idx]
random.shuffle(y)

In [33]:
y.shape

(244,)

In [56]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 77)
print(x_train.shape)
print(x_test.shape)

(207, 1055)
(37, 1055)


In [57]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, metrics, model_selection

# rf = RandomForestClassifier(random_state = 1)
rf = svm.SVC()
score_list = cross_val_score(rf, X, y, cv = 10)

In [58]:
result = list(map(lambda x: '{score:.2f}'.format(score = x), score_list))
print(result)

['0.64', '0.68', '0.68', '0.64', '0.67', '0.67', '0.67', '0.67', '0.67', '0.67']


In [59]:
print("평균 정답률 = ", sum(score_list) / len(score_list))

평균 정답률 =  0.664


In [60]:
# validation_scores = []

# k = 10
# bins = 212 // k # (1) bins : 한 폴드에 들어갈 샘플의 개수

# for i in range(k) :

#   # (2) 검증 폴드 생성
#   start = i*bins
#   end = (i+1)*bins
#   val_fold = x_train[start:end]
#   val_target = y_train[start:end]

#   # (3) 훈련 폴드 생성
#   train_index = list(range(0,start)) + list(range(end,212))
#   train_fold = x_train[train_index]
#   train_target = y_train[train_index]

#   # (4) 훈련폴드 및 검증폴드 전처리
#   train_mean = np.mean(train_fold, axis=0)
#   train_std = np.std(train_fold, axis=0)
#   train_fold_scaled = (train_fold - train_mean) / train_std
#   val_fold_scaled = (val_fold - train_mean) / train_std

#   lyr = SingleLayer(l2=0.001)
#   lyr.fit(train_fold_scaled, train_target,epochs=100)
#   score = lyr.score(val_fold_scaled, val_target)
  
#   #(5) 폴드별 성능 기록
#   validation_scores.append(score)

In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 로지스틱 회귀모델
lr = LogisticRegression(random_state=0)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
y_pred_probability = lr.predict_proba(x_test)[:,1]

# 성능 평가
print("accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.68
Precision : 0.676
Recall : 1.000
F1 : 0.806


In [62]:
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse=True)

In [63]:
invert_index_vectorizer = {v: k for k, v in index_vectorizer.vocabulary_.items()}
print(str(invert_index_vectorizer)[:100]+'..')

{347: '맛/Noun', 174: '난건/Noun', 730: '오전/Noun', 977: '품절/Noun', 34: '가성/Noun', 511: '비는/Verb', 883: ..


In [64]:
for coef in coef_pos_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])

기다림/Noun 0.7085642504410312
좀/Noun 0.6202000448358057
서비스/Noun 0.4833220863320478
이런/Modifier 0.36507197591481844
별로/Noun 0.3606640771773122
가격/Noun 0.34465645525998306
괜찮은/Adjective 0.3413880878004461
너무/Adverb 0.3369871581676453
추천/Noun 0.33594143911806906
존맛/Noun 0.33059684356788344
근데/Adverb 0.3289672798630903
친절합니다/Adjective 0.3249849492578755
좋다/Adjective 0.321884379649418
친절하신/Adjective 0.30561787568335547
ㅎㅎㅎㅎ/KoreanParticle 0.30561787568335547
먹었는데/Verb 0.29738355999901445
하시는/Verb 0.2843463634139457
좋음/Adjective 0.27065077574540375
끝났다/Verb 0.2703567480259844
많음/Adjective 0.2679314504192879


In [65]:
for coef in coef_pos_index[-20:]:
    print(invert_index_vectorizer[coef[1]], coef[0])

좋아서/Adjective -0.39220385601297963
맛집/Noun -0.40068464753016697
시키지/Verb -0.4056698578442875
좋고/Adjective -0.4076695217886233
만족/Noun -0.40800223066958574
잇게/Verb -0.4105293320991558
소/Modifier -0.4264487694097183
저렴한/Adjective -0.4331345253443228
예뻐요/Adjective -0.43580154803026677
이쁘고/Adjective -0.4397981035818312
깔끔하고/Adjective -0.44965974870470776
하면/Verb -0.457830267193406
두/Determiner -0.460492054080525
친절하고/Adjective -0.4765195631191979
괜찮습니다/Adjective -0.5139619692319809
좋아해요/Adjective -0.5443950891733853
업음/Verb -0.5443950891733853
아쉽네요/Adjective -0.5494237596442767
이/Determiner -0.5947905048987491
맛있어요/Adjective -0.7618130125993119


In [67]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import numpy as np

In [68]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)


In [69]:
clf = KNeighborsClassifier(n_neighbors = 10)
scoring = 'accuracy'
score = cross_val_score(clf, X, y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

[0.76       0.52       0.68       0.64       0.5        0.625
 0.70833333 0.58333333 0.79166667 0.75      ]


In [70]:
# kNN Score
round(np.mean(score)*100, 2)

65.58

In [71]:
#사이킷런에 있음
clf = DecisionTreeClassifier()
scoring = 'accuracy'
score = cross_val_score(clf, X, y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

[0.6        0.48       0.56       0.52       0.54166667 0.58333333
 0.66666667 0.58333333 0.66666667 0.625     ]


In [72]:
# decision tree Score
round(np.mean(score)*100, 2)

58.27

In [73]:
clf = RandomForestClassifier(n_estimators=13)
scoring = 'accuracy'
score = cross_val_score(clf, X, y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

[0.72       0.56       0.6        0.64       0.5        0.58333333
 0.66666667 0.625      0.83333333 0.66666667]


In [74]:
# Random Forest Score
round(np.mean(score)*100, 2)

63.95

In [75]:
clf = GaussianNB()
scoring = 'accuracy'
score = cross_val_score(clf, X, y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

[nan nan nan nan nan nan nan nan nan nan]


In [76]:
clf = SVC()
scoring = 'accuracy'
score = cross_val_score(clf, X, y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)

[0.76       0.56       0.68       0.64       0.5        0.625
 0.70833333 0.625      0.83333333 0.75      ]


In [77]:
round(np.mean(score)*100,2)

66.82